In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import io

In [2]:
#Get the info of all AMs

All_AMs = {'IIC': 'https://www.cnmv.es/Portal/Consultas/ListadoEntidad.aspx?id=2&tipoent=0',
          'UCITS sucursal': 'https://www.cnmv.es/Portal/Consultas/ListadoEntidad.aspx?id=2&tipoent=1',
          'UCITS libre prestación': 'https://www.cnmv.es/Portal/Consultas/ListadoEntidad.aspx?id=2&tipoent=3',
          'FIA sucursal': 'https://www.cnmv.es/Portal/Consultas/ListadoEntidad.aspx?id=2&tipoent=5',
          'FIA libre prestación': 'https://www.cnmv.es/Portal/Consultas/ListadoEntidad.aspx?id=2&tipoent=6'}

list_keys = list(All_AMs.keys())
for i_type in list_keys:

    main_url = All_AMs[i_type]
    soup = BeautifulSoup(requests.get(main_url).content, 'lxml')
    soup_list_AM = soup.find_all('li', {'class': 'blocks-single'})

    AM_link=[]
    AM_name=[]
    AM_code=[]
    for i in range(len(soup_list_AM)):
        AM_link += ['https://www.cnmv.es/Portal/Consultas/'+soup_list_AM[i].find_all('a', href=True)[0]['href']]
        AM_name += [x.text.strip() for x in soup_list_AM[i].find_all('span')]
        AM_code += [soup_list_AM[i].find_all('li', {'class': 'padding-r resumen'})[0].text.strip().replace('Número y fecha de registro oficial:','').replace(' ','').split('-')[0]]

    aux_df = pd.DataFrame(data={'Type': i_type, 'AM Name': AM_name, 'AM Code': AM_code, 'Global Data URL': AM_link})
    if i_type == list_keys[0]:
        Global_df = aux_df
        
    else:
        Global_df = pd.concat([Global_df, aux_df], ignore_index=True)
Global_df

,Type,AM Name,AM Code,Global Data URL
0,IIC,"360 CORA SGIIC, S.A.",276,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...
1,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...
2,IIC,"ABACO CAPITAL, SGIIC, S.A.",238,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...
3,IIC,"ABANTE ASESORES GESTION, SGIIC, S.A.",194,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...
4,IIC,"ACACIA INVERSION, SGIIC, S.A.",234,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...
5,IIC,"ACCI CAPITAL INVESTMENTS, SGIIC, S.A.",254,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...
6,IIC,"AFI INVERSIONES GLOBALES, SGIIC, S.A.",274,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...
7,IIC,"ALALUZ CAPITAL, SGIIC, S.A.",245,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...
8,IIC,"ALANTRA EQMC ASSET MANAGEMENT, SGIIC, S.A.",260,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...
9,IIC,"ALANTRA MULTI ASSET, SGIIC, S.A.",230,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...


In [3]:
#Download all funds URLs

for i_AM in range(len(Global_df)):
    print(i_AM/len(Global_df))

    try:
        #Get in the Global Display of the AM and go to the tab 'Fondos Gestionados'
        soup = BeautifulSoup(requests.get(Global_df.iloc[i_AM]['Global Data URL']).content, 'lxml')
        soup_link_funds = soup.find('div', {'class': 'NavegApdo'}).find_all('a', href=True)
        for i_links in soup_link_funds:
            if i_links.text=='Fondos gestionados':
                link_funds = i_links['href']

        link_funds = '/'.join(Global_df.iloc[i_AM]['Global Data URL'].split('/')[:-1]) + '/' + link_funds

        #Once inside the tab 'Fondos Gestionados', go to the table 'Fondos de inversión gestionados' and scrape all funds and URLs 
        soup = BeautifulSoup(requests.get(link_funds).content, 'lxml')
        soup_link_funds = soup.find_all('div', {'id': 'id_tabladatos'})

        
        for i in range(len(soup_link_funds)):
            if 'Fondos de inversión gestionados' in soup_link_funds[i].text:
                soup_link_funds = soup_link_funds[i]
                break
                
        name_funds = []
        list_URL_funds = []
        for i_fund in soup_link_funds.find_all('a', href=True):
            name_funds += [i_fund.text]
            list_URL_funds += ['/'.join(Global_df.iloc[i_AM]['Global Data URL'].split('/')[:-1]) + '/' + i_fund['href']]
            
        list_code_funds = []
        for i_fund in soup_link_funds.find_all('td', {'data-th':'Nº Registro oficial'}):
            list_code_funds += [i_fund.text]

        aux_df_funds = pd.DataFrame(data={'AM Name': Global_df.iloc[i_AM]['AM Name'], 'Fund Name': name_funds, 'Fund Code': list_code_funds, 'Fund Global URL': list_URL_funds})
    
    except:
        aux_df_funds = pd.DataFrame(data={'AM Name': Global_df.iloc[i_AM]['AM Name'], 'Fund Name': [np.nan], 'Fund Code': [np.nan], 'Fund Global URL': [np.nan]})
        
    if i_AM == 0:
        Funds_df = aux_df_funds
        
    else:
        Funds_df = pd.concat([Funds_df, aux_df_funds], ignore_index=True)    

Global_df = pd.merge(Global_df, Funds_df, how='left', on=['AM Name'])
Global_df['Fund ID'] = Global_df.agg('{0[AM Code]}-{0[Fund Code]}'.format, axis=1)
Global_df

0.0
0.0013717421124828531
0.0027434842249657062
0.00411522633744856
0.0054869684499314125
0.006858710562414266
0.00823045267489712
0.009602194787379973
0.010973936899862825
0.012345679012345678
0.013717421124828532
0.015089163237311385
0.01646090534979424
0.01783264746227709
0.019204389574759947
0.0205761316872428
0.02194787379972565
0.023319615912208505
0.024691358024691357
0.02606310013717421
0.027434842249657063
0.02880658436213992
0.03017832647462277
0.03155006858710562
0.03292181069958848
0.03429355281207133
0.03566529492455418
0.037037037037037035
0.038408779149519894
0.039780521262002745
0.0411522633744856
0.04252400548696845
0.0438957475994513
0.04526748971193416
0.04663923182441701
0.04801097393689986
0.04938271604938271
0.05075445816186557
0.05212620027434842
0.053497942386831275
0.05486968449931413
0.056241426611796985
0.05761316872427984
0.05898491083676269
0.06035665294924554
0.06172839506172839
0.06310013717421124
0.0644718792866941
0.06584362139917696
0.06721536351165981

0.5775034293552812
0.578875171467764
0.5802469135802469
0.5816186556927297
0.5829903978052127
0.5843621399176955
0.5857338820301783
0.5871056241426612
0.588477366255144
0.5898491083676269
0.5912208504801097
0.5925925925925926
0.5939643347050755
0.5953360768175583
0.5967078189300411
0.598079561042524
0.5994513031550068
0.6008230452674898
0.6021947873799726
0.6035665294924554
0.6049382716049383
0.6063100137174211
0.607681755829904
0.6090534979423868
0.6104252400548696
0.6117969821673526
0.6131687242798354
0.6145404663923183
0.6159122085048011
0.6172839506172839
0.6186556927297668
0.6200274348422496
0.6213991769547325
0.6227709190672154
0.6241426611796982
0.6255144032921811
0.6268861454046639
0.6282578875171467
0.6296296296296297
0.6310013717421125
0.6323731138545954
0.6337448559670782
0.635116598079561
0.6364883401920439
0.6378600823045267
0.6392318244170097
0.6406035665294925
0.6419753086419753
0.6433470507544582
0.644718792866941
0.6460905349794238
0.6474622770919067
0.6488340192043895

,Type,AM Name,AM Code,Global Data URL,Fund Name,Fund Code,Fund Global URL
0,IIC,"360 CORA SGIIC, S.A.",276,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"CODEX GLOBAL FUND, FI",5461,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...
1,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"A&G RENTA FIJA CORTO PLAZO, FI",3989,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...
2,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"GLOBAL MANAGERS FUNDS, FI",3072,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...
3,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"GREDOS BOLSA EURO, FI",4881,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...
4,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"GREDOS BOLSA INTERNACIONAL, FI",4883,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...
5,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"GREDOS MODERADO, FI",4882,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...
6,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"GREDOS RENTA FIJA, FI",5389,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...
7,IIC,"ABACO CAPITAL, SGIIC, S.A.",238,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,ABACO GLOBAL VALUE OPPORTUNITIES FI,4827,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...
8,IIC,"ABACO CAPITAL, SGIIC, S.A.",238,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"ABACO RENTA FIJA MIXTA GLOBAL, FI",4474,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...
9,IIC,"ABANTE ASESORES GESTION, SGIIC, S.A.",194,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"ABANTE ASESORES GLOBAL, FI",2562,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...


In [7]:
Global_df.to_excel('CNMV_Global_Scrape_v2.xlsx')

In [13]:
#Download all last XML funds available

for i_Fund in range(len(Global_df)):
    print(i_Fund/len(Global_df))
    
    if np.isnan(Global_df.iloc[i_Fund]['Fund Code']): #Check if there is registered a fund
        continue
    
    else:
        
        try:
            
            #Get in the Global Display of the Fund and go to the tab 'Información pública periódica'
            soup = BeautifulSoup(requests.get(Global_df.iloc[i_Fund]['Fund Global URL']).content, 'lxml')
            soup_portfolio = soup.find('div', {'class': 'NavegApdo'}).find_all('a', href=True)

            for i_links in soup_portfolio:
                if i_links.text=='Información pública periódica':
                    link_portfolio = i_links['href']

            link_portfolio = '/'.join(Global_df.iloc[i_Fund]['Global Data URL'].split('/')[:-1]) + '/' + link_portfolio

            #Once inside the tab 'Información pública periódica', go to the table and scrape the first available XML in the table (most recent info)
            soup = BeautifulSoup(requests.get(link_portfolio).content, 'lxml')
            soup_portfolio = soup.find_all('div', {'id': 'id_tabladatos'})

            for i in range(len(soup_portfolio)):
                if 'Información pública periódica' in soup_portfolio[i].text:
                    soup_portfolio = soup_portfolio[i]
                    break

            link_download_port = soup_portfolio.find_all('tr')[1].find_all('a') #0 correspond to headers, 1 correspond to most recent data
            for i in range(len(link_download_port)): 
                if 'XBRL' in link_download_port[i]['href']:
                    link_download_port = link_download_port[i]['href']
                    break

            link_download_port = 'https://www.cnmv.es/Portal/Consultas' + link_download_port.replace('..','')

            #Save Fund XML
            with io.open(Global_df.iloc[i_Fund]['Fund ID']+'.xml', "w", encoding="utf-8") as f:
                f.write(requests.get(link_download_port).text)
                
            print('OK')
            
        except:
            
            print('Error')
            continue

0.0
OK
0.0004366812227074236
OK
0.0008733624454148472
OK
0.0013100436681222707
OK
0.0017467248908296944
OK
0.002183406113537118
OK
0.0026200873362445414
OK
0.003056768558951965
OK
0.0034934497816593887
OK
0.003930131004366812
OK
0.004366812227074236
OK
0.00480349344978166
OK
0.005240174672489083
OK
0.005676855895196507
OK
0.00611353711790393
OK
0.006550218340611353
OK
0.0069868995633187774
OK
0.007423580786026201
OK
0.007860262008733625
OK
0.008296943231441048
OK
0.008733624454148471
OK
0.009170305676855894
OK
0.00960698689956332
OK
0.010043668122270743
OK
0.010480349344978166
OK
0.010917030567685589
OK
0.011353711790393014
OK
0.011790393013100437
OK
0.01222707423580786
OK
0.012663755458515284
OK
0.013100436681222707
OK
0.013537117903930132
OK
0.013973799126637555
OK
0.014410480349344978
OK
0.014847161572052401
OK
0.015283842794759825
0.01572052401746725
OK
0.016157205240174673
OK
0.016593886462882096
OK
0.01703056768558952
OK
0.017467248908296942
OK
0.017903930131004366
OK
0.018340611

OK
0.15589519650655023
OK
0.15633187772925763
OK
0.15676855895196506
OK
0.1572052401746725
OK
0.15764192139737992
OK
0.15807860262008733
OK
0.15851528384279476
OK
0.1589519650655022
OK
0.15938864628820962
OK
0.15982532751091702
OK
0.16026200873362445
OK
0.16069868995633188
OK
0.1611353711790393
OK
0.1615720524017467
OK
0.16200873362445414
OK
0.16244541484716157
OK
0.162882096069869
OK
0.1633187772925764
OK
0.16375545851528384
OK
0.16419213973799127
OK
0.1646288209606987
OK
0.16506550218340613
OK
0.16550218340611353
OK
0.16593886462882096
OK
0.1663755458515284
OK
0.16681222707423582
OK
0.16724890829694322
OK
0.16768558951965065
OK
0.16812227074235808
OK
0.1685589519650655
OK
0.16899563318777291
OK
0.16943231441048034
OK
0.16986899563318777
OK
0.1703056768558952
OK
0.1707423580786026
OK
0.17117903930131004
OK
0.17161572052401747
OK
0.1720524017467249
OK
0.17248908296943233
OK
0.17292576419213973
OK
0.17336244541484716
OK
0.1737991266375546
OK
0.17423580786026202
OK
0.17467248908296942
OK

OK
0.31572052401746725
OK
0.31615720524017465
OK
0.3165938864628821
OK
0.3170305676855895
OK
0.3174672489082969
OK
0.3179039301310044
OK
0.3183406113537118
OK
0.31877729257641924
OK
0.31921397379912664
OK
0.31965065502183404
OK
0.3200873362445415
OK
0.3205240174672489
OK
0.32096069868995636
OK
0.32139737991266376
OK
0.32183406113537116
OK
0.3222707423580786
OK
0.322707423580786
OK
0.3231441048034934
OK
0.3235807860262009
OK
0.3240174672489083
OK
0.32445414847161574
OK
0.32489082969432315
OK
0.32532751091703055
OK
0.325764192139738
OK
0.3262008733624454
OK
0.3266375545851528
OK
0.32707423580786027
OK
0.32751091703056767
OK
0.32794759825327513
OK
0.32838427947598253
OK
0.32882096069868993
OK
0.3292576419213974
OK
0.3296943231441048
OK
0.33013100436681225
OK
0.33056768558951966
OK
0.33100436681222706
OK
0.3314410480349345
OK
0.3318777292576419
OK
0.3323144104803493
OK
0.3327510917030568
OK
0.3331877729257642
OK
0.33362445414847164
OK
0.33406113537117904
OK
0.33449781659388644
OK
0.3349344

OK
0.4759825327510917
OK
0.4764192139737991
OK
0.47685589519650656
OK
0.47729257641921397
OK
0.4777292576419214
OK
0.4781659388646288
OK
0.47860262008733623
OK
0.4790393013100437
OK
0.4794759825327511
OK
0.4799126637554585
OK
0.48034934497816595
OK
0.48078602620087335
OK
0.4812227074235808
OK
0.4816593886462882
OK
0.4820960698689956
OK
0.48253275109170307
OK
0.4829694323144105
OK
0.4834061135371179
OK
0.48384279475982533
OK
0.48427947598253274
OK
0.4847161572052402
OK
0.4851528384279476
OK
0.485589519650655
OK
0.48602620087336246
OK
0.48646288209606986
OK
0.4868995633187773
OK
0.4873362445414847
OK
0.4877729257641921
OK
0.4882096069868996
OK
0.488646288209607
OK
0.4890829694323144
OK
0.48951965065502184
OK
0.48995633187772925
OK
0.4903930131004367
OK
0.4908296943231441
OK
0.4912663755458515
OK
0.49170305676855897
OK
0.49213973799126637
OK
0.4925764192139738
OK
0.49301310043668123
OK
0.49344978165938863
OK
0.4938864628820961
OK
0.4943231441048035
OK
0.4947598253275109
OK
0.4951965065502

OK
0.6397379912663755
OK
0.640174672489083
OK
0.6406113537117903
OK
0.6410480349344978
OK
0.6414847161572053
OK
0.6419213973799127
OK
0.6423580786026201
OK
0.6427947598253275
OK
0.643231441048035
OK
0.6436681222707423
OK
0.6441048034934498
OK
0.6445414847161572
OK
0.6449781659388646
OK
0.645414847161572
OK
0.6458515283842795
OK
0.6462882096069869
OK
0.6467248908296943
OK
0.6471615720524018
OK
0.6475982532751092
OK
0.6480349344978166
OK
0.648471615720524
OK
0.6489082969432315
0.6493449781659388
OK
0.6497816593886463
OK
0.6502183406113538
OK
0.6506550218340611
OK
0.6510917030567686
OK
0.651528384279476
OK
0.6519650655021834
OK
0.6524017467248908
Error
0.6528384279475983
OK
0.6532751091703056
OK
0.6537117903930131
OK
0.6541484716157205
OK
0.654585152838428
OK
0.6550218340611353
OK
0.6554585152838428
OK
0.6558951965065503
OK
0.6563318777292576
OK
0.6567685589519651
OK
0.6572052401746725
OK
0.6576419213973799
OK
0.6580786026200873
OK
0.6585152838427948
OK
0.6589519650655021
OK
0.65938864628

0.9109170305676856
0.911353711790393
0.9117903930131004
0.9122270742358078
0.9126637554585153
0.9131004366812228
0.9135371179039301
0.9139737991266376
0.914410480349345
0.9148471615720524
0.9152838427947598
0.9157205240174673
0.9161572052401746
0.9165938864628821
0.9170305676855895
0.9174672489082969
0.9179039301310044
0.9183406113537118
0.9187772925764193
0.9192139737991266
0.9196506550218341
0.9200873362445415
0.9205240174672489
0.9209606986899563
0.9213973799126638
0.9218340611353711
0.9222707423580786
0.9227074235807861
0.9231441048034934
0.9235807860262009
0.9240174672489083
0.9244541484716157
0.9248908296943231
0.9253275109170306
0.925764192139738
0.9262008733624454
0.9266375545851528
0.9270742358078603
0.9275109170305676
0.9279475982532751
0.9283842794759826
0.9288209606986899
0.9292576419213974
0.9296943231441048
0.9301310043668122
0.9305676855895196
0.9310043668122271
0.9314410480349345
0.9318777292576419
0.9323144104803494
0.9327510917030568
0.9331877729257642
0.9336244541484